In [33]:
from datasets import load_dataset
import pandas as pd
import json
from datasets import Dataset, DatasetDict

In [32]:
train = load_dataset("Atipico1/NQ-10k", split="train")
test = load_dataset("Atipico1/NQ", split="test")

In [37]:
dataset = load_dataset("Atipico1/NQ-10k")

Generating test split: 100%|██████████| 3610/3610 [00:00<00:00, 56912.43 examples/s]


In [38]:
dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answers', 'ctxs'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['question', 'answers', 'ctxs'],
        num_rows: 3610
    })
})

In [35]:
total = DatasetDict({'train': train, 'test': test})
total.push_to_hub("Atipico1/NQ-10k")

README.md: 100%|██████████| 553/553 [00:00<00:00, 1.90MB/s]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/NQ-10k/commit/f8dc1d1786d9e352eeda939e3a56e142be3457ce', commit_message='Upload dataset', commit_description='', oid='f8dc1d1786d9e352eeda939e3a56e142be3457ce', pr_url=None, pr_revision=None, pr_num=None)

In [113]:
df = pd.read_csv('popQA.tsv', sep='\t')
df.head(1)

,id,subj,prop,obj,subj_id,prop_id,obj_id,s_aliases,o_aliases,s_uri,o_uri,s_wiki_title,o_wiki_title,s_pop,o_pop,question,possible_answers
0,4222362,George Rankin,occupation,politician,1850297,22,2834605,"[""George James Rankin""]","[""political leader"",""political figure"",""polit....",http://www.wikidata.org/entity/Q5543720,http://www.wikidata.org/entity/Q82955,George Rankin,Politician,142,25692,What is George Rankin's occupation?,"[""politician"", ""political leader"", ""political ..."


In [114]:
with open("/home/seongilpark/finetune_rag/contriever_results.jsonl", "r") as f:
    results = [json.loads(line) for line in f.readlines()]

In [115]:
pd.DataFrame(results).head(1)

,id,question,answers,ctxs
0,4222362,What is George Rankin's occupation?,"[politician, political leader, political figur...","[{'id': '11701803', 'title': 'George Rankin', ..."


In [116]:
concat = pd.merge(df, pd.DataFrame(results), left_on='id', right_on='id').drop(columns=['question_y', "answers"]).rename(columns={'question_x': 'question', "possible_answers": "answers"})

In [127]:
dataset = Dataset.from_pandas(concat)


In [128]:
dataset = dataset.train_test_split(train_size=10_000, seed=42)

In [132]:
for row in dataset["train"]:
    print(row["ctxs"])
    break

[{'hasanswer': False, 'id': '13995485', 'score': '1.6310263', 'text': 'Escape Artists Escape Artists Productions, LLC, commonly known as Escape Artists and distinct from Escape Artists, Inc. (pod caster), is an independently financed motion picture and television production company with a first look non-exclusive deal at Sony Pictures Entertainment, headed by partners Steve Tisch, Todd Black, and Jason Blumenthal. In 2001, Todd Black and Jason Blumenthal’s Black & Blu merged with the Steve Tisch Company to form Escape Artists. The first produced movie under the Escape Artists banner was "A Knight\'s Tale", starring Heath Ledger in 2001. In the fall of 2005, Escape Artists released "The Weather Man", directed by', 'title': 'Escape Artists'}, {'hasanswer': False, 'id': '18332725', 'score': '1.6303782', 'text': 'Tommy Rettig, and Brian Keith. The announcers were Jack McCoy and Elliott Lewis. Escape (1950 TV series) Escape was a 30-minute live American dramatic anthology television series 

In [129]:
dataset.push_to_hub("Atipico1/popQA")

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.24s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/popQA/commit/3586e3709ae7947c9629784cfc0705786eec7f0b', commit_message='Upload dataset', commit_description='', oid='3586e3709ae7947c9629784cfc0705786eec7f0b', pr_url=None, pr_revision=None, pr_num=None)

In [18]:
new_dataset["train"]["ctxs"][0]

[{'id': 679735,
  'text': '"Kodoku no Akatsuki" (孤独のあかつき, "The Solitude of Dawn") (also known by its French title "La Solitude de l\'aube") is a song by Japanese musician Ringo Sheena. It was released as one of the A-sides of her 14th single, along with the song "Irohanihoheto", released on May 27, 2013. The release date was the 15th anniversary of the release of Sheena\'s debut single "Kōfukuron" in 1998.',
  'hasanswer': False},
 {'id': 1944639,
  'text': "Doctor Who at the BBC Radiophonic Workshop Volume 1: The Early Years 1963–1969 was the first in a series of compilations of Doctor Who material recorded by the BBC Radiophonic Workshop. Compiled and remastered by Mark Ayres, the album features mostly sound effects and atmospheres from the first six years of the programme. Although some incidental music tracks do appear, most of the album's content is by original Doctor Who sound effects creator Brian Hodgson. The compilation also features three Radiophonic Workshop realisations of 

In [14]:
new_dataset = dataset.train_test_split(test_size=4282, seed=42)

In [16]:
new_dataset.push_to_hub("Atipico1/PopQA")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.65s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/PopQA/commit/08f0f594f1aa7342667bc6ed6c90368d048d3e76', commit_message='Upload dataset', commit_description='', oid='08f0f594f1aa7342667bc6ed6c90368d048d3e76', pr_url=None, pr_revision=None, pr_num=None)

In [19]:
from train import formatting_for_original

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [77]:
akari = load_dataset("akariasai/PopQA")

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [79]:
for row in akari["test"]:
    if not row["possible_answers"]:
        print(row["question"])

In [80]:
akari

DatasetDict({
    test: Dataset({
        features: ['id', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'question', 'possible_answers'],
        num_rows: 14267
    })
})

In [56]:
import pandas as pd

df_akari = pd.DataFrame(akari["test"])
dataset = load_dataset("Atipico1/PopQA")
df_hf_train = pd.DataFrame(dataset["train"])
df_hf_test = pd.DataFrame(dataset["test"])
new_df_hf_train = pd.merge(df_hf_train, df_akari, on=["id"], how="left")
new_df_hf_train.drop(columns=["question_y"], inplace=True)
new_df_hf_train.rename(columns={"question_x": "question", "possible_answers":"answers"}, inplace=True)
new_df_hf_test = pd.merge(df_hf_test, df_akari, on=["id"], how="left")
new_df_hf_test.drop(columns=["question_y"], inplace=True)
new_df_hf_test.rename(columns={"question_x": "question", "possible_answers":"answers"}, inplace=True)

In [57]:
from datasets import DatasetDict

final = DatasetDict({"train": Dataset.from_pandas(new_df_hf_train), "test": Dataset.from_pandas(new_df_hf_test)})

In [59]:
final.push_to_hub("Atipico1/PopQA")

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.26s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/PopQA/commit/4b229215976ff0e532df636ff94b60119d7cb674', commit_message='Upload dataset', commit_description='', oid='4b229215976ff0e532df636ff94b60119d7cb674', pr_url=None, pr_revision=None, pr_num=None)

In [81]:
final

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'ctxs', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'answers'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['id', 'question', 'ctxs', 'subj', 'prop', 'obj', 'subj_id', 'prop_id', 'obj_id', 's_aliases', 'o_aliases', 's_uri', 'o_uri', 's_wiki_title', 'o_wiki_title', 's_pop', 'o_pop', 'answers'],
        num_rows: 4282
    })
})

In [24]:
dataset = load_dataset("Atipico1/popQA")

In [25]:
from ast import literal_eval
dataset["train"] = dataset["train"].map(lambda x: {"answers": literal_eval(x["answers"])})
dataset["test"] = dataset["test"].map(lambda x: {"answers": literal_eval(x["answers"])})

Map:   0%|          | 0/4267 [00:00<?, ? examples/s]

Map: 100%|██████████| 4267/4267 [00:00<00:00, 6164.21 examples/s]


In [31]:
dataset.push_to_hub("Atipico1/popQA")

Uploading the dataset shards: 100%|██████████| 1/1 [00:04<00:00,  4.63s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/Atipico1/popQA/commit/39d1f9de940475fbf588c1d1c49f2fbabb49f477', commit_message='Upload dataset', commit_description='', oid='39d1f9de940475fbf588c1d1c49f2fbabb49f477', pr_url=None, pr_revision=None, pr_num=None)

In [13]:
from src.utils import normalize_question

num_contexts = 5
def formatting_for_original(example):
    output_texts = []
    for i in range(len(example['question'])):
        q = normalize_question(example['question'][i])
        if num_contexts == 0:
            text = f"### Q: {q}\n### A: {example['answers'][i][0]}</s>"
        else:
            ctxs = "\n".join([f"Doc {idx}: {ctx['text']}" for idx, ctx in enumerate(example["ctxs"][i][:num_contexts])])
            text = f"### Background:\n{ctxs}\n\n### Q: {q}\n### A: {example['answers'][i][0]}</s>"
        output_texts.append(text)
    return output_texts
output = formatting_for_original(dataset[:])

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-hf")

/data/seongilpark/conda_envs/ft/lib/python3.10/site-packages/transformers/utils/hub.py:123: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [138]:
out = tokenizer(output, return_length=True)

In [139]:
max(out['length'])

1367

In [36]:
1024+512

1536